# wav2lip 單獨運行含Gradio界面範例 (wav2lip_run.ipynb)

by Jack OmniXRI, 2024/12/12

本範例為簡化版，運行前必需執行過 wav2lip.ipynb，下載及轉換好相關檔案，包括:  
*  notebook_utils.py, pip_helper.py, cmd_helper.py
*  gradio_helper.py, ov_inference.py, ov_wav2lip_helper.py
*  data_audio_sun_5s.wav, data_vidio_sun_5s.mp4 
*  \wav2lip 下所有模組
*  \models\face_detection.bin & xml,  \models\wav2lip.bin & xml
  

In [1]:
from notebook_utils import device_widget

# 設定下拉式選單，以選擇推論裝置 (CPU, GPU, NPU, AUTO) (這裡的GPU是指Intel內顯）
device = device_widget() 
device

Dropdown(description='Device:', index=3, options=('CPU', 'GPU', 'NPU', 'AUTO'), value='AUTO')

In [2]:
import os
import sys
from pathlib import Path

#print(os.path.abspath('.'))
sys.path.append('./wav2lip') # 為了讓系統找得到 wav2lip 相關函式，所以手動加入相關路徑

OV_FACE_DETECTION_MODEL_PATH = Path("models/face_detection.xml") # 指定人臉偵測模型路徑
OV_WAV2LIP_MODEL_PATH = Path("models/wav2lip.xml") # 指定 wav2lip 模型路徑

In [3]:
from ov_inference import ov_inference

# 確認存放輸出結果路徑是否存在，若否則建立 \results 路徑
if not os.path.exists("results"):
    os.mkdir("results")

# 使用 OpenVINO 進行推論
ov_inference(
    "data_video_sun_5s.mp4", # 指定原始影片檔案
    "data_audio_sun_5s.wav", # 指定聲音檔案
    face_detection_path=OV_FACE_DETECTION_MODEL_PATH, # 指定人臉偵測模型路徑
    wav2lip_path=OV_WAV2LIP_MODEL_PATH, # 指定 wav2lip 模型路徑
    inference_device=device.value, # 指定推論裝置
    outfile="results/result_voice.mp4", # 輸出結果檔案名稱
)

Reading video frames...
Number of frames available for inference: 125


C:\Users\omnixri\openvino_notebooks\2024_4\openvino_notebooks\notebooks\digital_human\Wav2Lip\audio.py:100: FutureWarning: Pass sr=16000, n_fft=800 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,


(80, 405)
Length of mel chunks: 123


  0%|                                                                                    | 0/1 [00:00<?, ?it/s]

face_detect_ov images[0].shape:  (768, 576, 3)



100%|████████████████████████████████████████████████████████████████████████████| 8/8 [02:27<00:00, 18.44s/it]


Model loaded


100%|███████████████████████████████████████████████████████████████████████████| 1/1 [02:31<00:00, 151.29s/it]


'results/result_voice.mp4'

## 使用 Gradio 產生互動界面

除在欄位上顯示操作界面外，亦可在 http://127.0.0.1:7860 (http://localhost:7860) 以網頁方式呈現。  
在操作時，最下方有預設樣本(Examples)，點擊後會自動載入輸入端，當按下「Submit」後，即會開始轉換影片。  
這裡亦可自行選擇檔案上傳影音檔案或以網路攝影機拍攝及錄音，再進行轉換。

In [ ]:
from gradio_helper import make_demo

demo = make_demo(fn=ov_inference)

try:
    demo.queue().launch(debug=True)
except Exception:
    demo.queue().launch(debug=True, share=True)

IMPORTANT: You are using gradio version 4.26.0, however version 4.44.1 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
